<a href="https://colab.research.google.com/github/kuryu6116/AITalker/blob/Master/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **初期設定**

In [5]:
#Talk_APIのキー設定
import os
API_Key = "" #@param {type:"string",title:"キー入力"}

#GoogleDriveの連携
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 形態素解析(Mecab)

In [ ]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.996.5
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a
!pip install unidic-lite

# BERT(京都大学モデル)

In [ ]:
!pip install torch
!pip install ipykernel
!pip install pandas
!pip install keras
!pip install sklearn
!pip install keras-bert
!pip install transformers
!pip install pyknp
from google.colab import drive
drive.mount('/content/drive')
!sudo apt update
!sudo apt upgrade
!sudo apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub

!wget wget http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.2.89-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804_10.2.89-1_amd64.deb
!sudo apt update

!sudo apt install cuda cuda-drivers
!sudo reboot

!rm cuda-repo-ubuntu1804_10.2.89-1_amd64.deb

!export PATH="/usr/local/cuda/bin:$PATH"
!export LD_LIBRARY_PATH="/usr/local/cuda/lib64:$LD_LIBRARY_PATH"

In [ ]:
import MeCab
import subprocess
import re
import torch
from transformers import BertTokenizer, BertForMaskedLM, BertConfig
import numpy as np


def bert(_text):
  return_text=""
  config = BertConfig.from_json_file('/content/drive/My Drive/bert/config.json')
  model = BertForMaskedLM.from_pretrained('/content/drive/My Drive/bert/pytorch_model.bin', config=config)
  bert_tokenizer = BertTokenizer('/content/drive/My Drive/bert/vocab.txt',
  do_lower_case=False, do_basic_tokenize=False)
  cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
  path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                            shell=True).communicate()[0]).decode('utf-8')

  m=MeCab.Tagger("-d {0}".format(path))
  mecab=m.parse(_text)
  lines = mecab.split('\n')
  #各行ごとに文章の構成単位に分解
  items = (re.split('[\t]',line) for line in lines)
  mecab_list=[]
  tokenized_text=[]
  [tokenized_text.append(item[0]) for item in items]
  print(tokenized_text)
  #形態素解析した結果を表示
  tokenized_text.insert(0, '[CLS]')
  tokenized_list=[]
  count=0
  masked_index = 0
  for text in tokenized_text:
      count+=1
      if text=='何':
          tokenized_list.append('[MASK]')
          masked_index=count-1
      elif '。' in text:
          tokenized_list.append(text)
          tokenized_list.append('[SEP]')
          count+=2
      elif text=='EOS':
          count+=1
      else:
          tokenized_list.append(text)
  tokens = bert_tokenizer.convert_tokens_to_ids(tokenized_list)
  tokens_tensor = torch.tensor([tokens])
  print(tokenized_list)

  model.eval()
  
  tokens_tensor = tokens_tensor.to('cuda')
  model.to('cuda')
  
  with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]
  
    _, predicted_indexes = torch.topk(predictions[0, masked_index], k=3)
    predicted_tokens = bert_tokenizer.convert_ids_to_tokens(predicted_indexes.tolist())
    
    for list_text in predicted_tokens:
      return_text+=list_text+','
    return return_text

# **チャットボットの作成**

In [6]:
import requests
from IPython.display import Image
def talk_api(message):
    talk_url = "https://api.a3rt.recruit-tech.co.jp/talk/v1/smalltalk"
    payload = {"apikey": API_Key, "query": message}
    response = requests.post(talk_url, data=payload)
    try:
        return response.json()["results"][0]["reply"]
    except:
        print(response.json())
        return "ごめんなさい。もう一度教えて下さい。"

# **感情極性辞書の作成**

In [ ]:
!pip install gensim

In [ ]:
#import gensim.models.keyedvectors as word2vec
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/bert/model.vec',binary=False)

#「非常にポジティブな単語」と「非常にネガティブな単語」を任意で指定
posi_list = ['優れる', '良い','喜ぶ','褒める', 'めでたい','賢い','善い', '適す','天晴',
 '祝う', '功績','賞','嬉しい','喜び','才知','徳', '才能','素晴らしい','芳しい','称える',
 '適切','崇める','助ける','抜きんでる','清水','雄雄しい','仕合せ','幸い','吉兆','秀でる']

nega_list = ['悪い', '死ぬ', '病気', '酷い', '罵る', '浸ける', '卑しい',
 '下手', '苦しむ', '苦しい', '付く', '厳しい', '難しい', '殺す', '難い', '荒荒しい',
 '惨い', '責める', '敵', '背く', '嘲る', '苦しめる', '辛い', '物寂しい', '罰', '不貞腐る',
 '寒い', '下らない', '残念']

def posi_nega_score(x):
    #ポジティブ度合いの判定
    posi = []
    for i in posi_list:
        try:
            n = model.similarity(i, x)
            posi.append(n)
        except:
            continue
    try:
        posi_mean = sum(posi)/len(posi)
    except:
        posi_mean = 0

    #ネガティブ度合いの判定
    nega = []
    for i in nega_list:
        try:
            n = model.similarity(i, x)
            nega.append(n)
        except:
            continue
    try:
        nega_mean = sum(nega)/len(nega)
    except:
        nega_mean = 0
    if posi_mean > nega_mean:
        return posi_mean
    if nega_mean > posi_mean:
        return -nega_mean
    else:
        return 0

In [ ]:
import MeCab
import subprocess
import re


def Emotion(_text):
  return_text=""
  result_num=0
  cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
  path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                            shell=True).communicate()[0]).decode('utf-8')

  m=MeCab.Tagger("-d {0}".format(path))
  mecab=m.parse(_text)
  lines = mecab.split('\n')
  #各行ごとに文章の構成単位に分解
  items = (re.split('[\t]',line) for line in lines)
  mecab_list=[]
  tokenized_text=[]
  [tokenized_text.append(item[0]) for item in items]
  #形態素解析した結果を表示
  #print(tokenized_text)
  for text in tokenized_text:
    if(text!='EOS' or text!=''):
      result_num+=posi_nega_score(text)
  return str(result_num)



# **API環境の構築**

In [ ]:
!pip install flask-ngrok
!pip install requests
!pip install pycurl
!pip uninstall Flask-SocketIO

In [ ]:
def OpenText(_text):
  __text=_text.split(':')
  e=__text[1].split('"')
  return e[1]

In [ ]:
import pycurl
from flask_ngrok import run_with_ngrok
from flask import Flask,jsonify
from flask import request

app = Flask(__name__)
app.config["JSON_AS_ASCII"] = False
run_with_ngrok(app)   #starts ngrok when the app is run
result_text=""
result=""


@app.route("/AI",methods=["POST"])
def AIResult():
  #Jsonを解析する処理
  result_text=OpenText(request.get_data(as_text=True))
  result=bert(result_text)

  return result

@app.route("/chat",methods=["POST"])
def ChatResult():
  #Jsonを解析する処理
  result_text=OpenText(request.get_data(as_text=True))
  result = talk_api(result_text)
    
  return result

@app.route("/emotion",methods=["POST"])
def EmotionResult():
  #Jsonを解析する処理
  result_text=OpenText(request.get_data(as_text=True))
  result = Emotion(result_text)

  return result

if __name__ == '__main__':
  app.run()